In [7]:
import numpy as np
import tensorflow as tf
import cv2
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import skimage
import os
from sklearn.metrics import mean_squared_error
from skimage import data, img_as_float
from skimage.restoration import denoise_nl_means
from skimage.measure import compare_ssim
from sklearn.metrics import mean_squared_error
from scipy.stats import pearsonr
from skimage.measure import compare_psnr,compare_mse
from scipy import ndimage, misc
from random import shuffle
import math
from keras.models import model_from_json, load_model
from keras.layers import *
from keras.models import Model
import re
from keras import regularizers
from keras.utils import plot_model
from keras.utils.vis_utils import plot_model
from keras import backend as K
from keras.callbacks import TensorBoard, ReduceLROnPlateau, CSVLogger, EarlyStopping, ModelCheckpoint
from keras.layers.core import Lambda
from keras.preprocessing.image import ImageDataGenerator
from keras.losses import mean_squared_error
import scipy.io as sio
import pickle
from random import randint
from utilities import *
from PIL import Image

from IPython.display import clear_output


In [3]:
#Convert rgb to gray
def rgb2gray(rgb):

    r, g, b = rgb[:,:,:,0], rgb[:,:,:,1], rgb[:,:,:,2]
    gray = 0.2989 * r + 0.5870 * g + 0.1140 * b

    return gray

#Read images from folder of given path
def read_images_from_folder(folder_path):
    images = []
    for filename in os.listdir(folder_path):
        img = cv2.imread(os.path.join(folder_path,filename),0)
        if img is not None:
            images.append(img)
    return np.array(images)
        

In [8]:
def resize_images(src_path,dest_path,sizeX,sizeY):
    images = []
    width,height = sizeX,sizeY
    for filename in os.listdir(src_path):
        img = cv2.imread(src_path + filename)
        if img is not None:
            images.append(img)
    cnt =1
    for img in images:
        im2 = cv2.resize(img,(width, height),interpolation = cv2.INTER_CUBIC)
        
        
        ext = ".png"
        cv2.imwrite(dest_path + "image" +  str(cnt) + ext,im2)
        cnt+=1

In [9]:
#Read images from folder
raw_images = read_images_from_folder('./data/oct_gt/')
raw_images = np.array(raw_images)

In [10]:
raw_images.shape

(17, 450, 900)

In [11]:
#Adding channel dimension
raw_image_data = np.reshape(raw_images,(raw_images.shape[0],raw_images.shape[1],raw_images.shape[2],1))

In [12]:
#Adding noise to images
def add_noise(images):
    
    noisy_set = []
    for image in images:

        for shape in xrange(2,10,1):

            for scale in xrange(2,10,1):

                noise = np.random.gamma(shape,scale,image.shape)
                
                noisy_img = noise + image
                noisy_img = 255.*((noisy_img - noisy_img.min())/(noisy_img.max()-noisy_img.min()))
                
                noisy_set.append((image,noisy_img))
                
        clear_output()
        print len(noisy_set)

        
    shuffle(noisy_set)
    return np.array(noisy_set)
   
#Shuffle the noisy image ground truth pair to randomize the noise distribution in the dataset
def expand_pair(noisy_set):   
    ground_truth=[]
    noisy_images = []
    for i in range(noisy_set.shape[0]):
        ground_truth.append(noisy_set[i][0].reshape((noisy_set[i][0].shape[0],noisy_set[i][0].shape[1],1)))
        #print( str(noisy_set[i][0].shape[0]) +" "+ str(noisy_set[i][0].shape[1]))
        noisy_images.append(noisy_set[i][1].reshape((noisy_set[i][1].shape[0],noisy_set[i][1].shape[1],1)))
    return np.array(ground_truth), np.array(noisy_images)

In [13]:
#Adding noise
noisy_set = add_noise(raw_image_data)
#for i in range(noisy_set.shape[0]):
    #cv2.imwrite("./data/oct_noisy/"+str(i)+".tif",noisy_set[i][1])    

1088


In [14]:
#Shuffling and adding noise to the dataset
ground_truth,noisy_images = expand_pair(noisy_set)
print (ground_truth.shape, noisy_images.shape, noisy_set.shape)

((1088, 450, 900, 1), (1088, 450, 900, 1), (1088, 2, 450, 900, 1))


In [16]:
def resize_images(images,size):
    resized_images = []
    
    for img in images:
        im2 = cv2.resize(img,size,interpolation = cv2.INTER_CUBIC)
        resized_images.append(im2)
    
    resized_images = np.array(resized_images)
    resized_images = np.reshape(resized_images,(resized_images.shape[0],resized_images.shape[1],resized_images.shape[2],1))
    
    return resized_images

ground_truth_rs = resize_images(ground_truth,(192,96))
noisy_images_rs = resize_images(noisy_images,(192,96))

In [17]:
noisy_images_rs.shape

(1088, 96, 192, 1)

In [18]:
#Saving ground truth and noisy images
for i in range(noisy_images_rs.shape[0]):
    cv2.imwrite("./data/oct_noisy_resized/"+str(i)+".tif",noisy_images_rs[i])  
    
for i in range(ground_truth_rs.shape[0]):
    cv2.imwrite("./data/oct_gt_resized/"+str(i)+".tif",ground_truth_rs[i])    

In [19]:
#Reading ground truth and noisy images from folders
ground_truth_rs = read_images_from_folder('./data/oct_gt_resized/')
ground_truth_rs = np.resize(ground_truth_rs,(ground_truth_rs.shape[0],ground_truth_rs.shape[1],ground_truth_rs.shape[2],1))

noisy_images_rs = read_images_from_folder('./data/oct_noisy_resized/')
noisy_images_rs = np.resize(noisy_images_rs,(noisy_images_rs.shape[0],noisy_images_rs.shape[1],noisy_images_rs.shape[2],1))

In [20]:
#Split into training and cross validation and normalizing
train_size = int(ground_truth_rs.shape[0]*0.9)
x_train = ground_truth_rs[0:train_size]/255.
x_train_noisy = noisy_images_rs[0:train_size]/255.
x_test = ground_truth_rs[train_size:]/255.
x_test_noisy = noisy_images_rs[train_size:]/255.
print (x_train_noisy.shape)
print (x_test_noisy.shape)

(979, 96, 192, 1)
(109, 96, 192, 1)


In [6]:
#lr_reducer = ReduceLROnPlateau(factor = np.sqrt(0.1), cooldown=0, patience=2, min_lr=0.5e-6)
#csv_logger = CSVLogger('./models/simple_cnn_autoencoder.csv')
#early_stopper = EarlyStopping(min_delta=0.001,patience=30)
model_checkpoint = ModelCheckpoint('./models/simple_cnn_oct.hdf5',monitor = 'loss', verbose = 1,save_best_only=True)

In [ ]:
x_train_noisy.shape

(666, 128, 256, 1)

In [3]:
#Defining the model

def get_simple_cnn_autoencoder_model(model_path=None):
    
    if(model_path is None):
        autoencoder = None
    else:
        autoencoder = read_model_json(model_path) 
    
    if(autoencoder is None):
        input_img = Input(shape=((None,None,1)))  # adapt this if using `channels_first` image data format

        x = Conv2D(64, (3, 3), activation='relu', padding='same')(input_img)
        x = MaxPooling2D((2, 2), padding='same')(x)
        x = Conv2D(64, (3, 3), activation='relu', padding='same')(x)
        encoded = MaxPooling2D((2, 2), padding='same')(x)

        x = Conv2D(64, (3, 3), activation='relu', padding='same')(encoded)
        x = UpSampling2D((2, 2))(x)
        x = Conv2D(64, (3, 3), activation='relu', padding='same')(x)
        x = UpSampling2D((2, 2))(x)
        
        decoded = Conv2D(1, (3, 3), activation='sigmoid', padding='same')(x)

        autoencoder = Model(input_img, decoded)
        
        
        autoencoder.compile(optimizer='Adam', loss='mean_squared_error',metrics = ['accuracy','mean_squared_error'])

    print (autoencoder.summary())
    return autoencoder

In [4]:
#Training the model
autoencoder = get_simple_cnn_autoencoder_model()


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, None, None, 1)     0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, None, None, 64)    640       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, None, None, 64)    0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, None, None, 64)    36928     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, None, None, 64)    0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, None, None, 64)    36928     
_________________________________________________________________
up_sampling2d_1 (UpSampling2 (None, None, None, 64)    0         
__________

In [78]:
plot_model(autoencoder, to_file='model.png',show_shapes='True',show_layer_names='True')

In [5]:
autoencoder.fit(x_train_noisy, x_train,
                epochs=500,
                batch_size=20,
                shuffle=True,
                validation_data=(x_test_noisy, x_test),
                callbacks=[TensorBoard(log_dir='/tmp/autoencoder', histogram_freq=0, write_graph=True), model_checkpoint])

NameError: name 'x_train_noisy' is not defined

In [21]:
#Function to get saved keras model
def read_model_json(jsonfilePath,h5filePath):
    try:
        json_file = open(jsonfilePath, 'r')
        print json_file
        loaded_model_json = json_file.read()
        json_file.close()
        print "hello"
        loaded_model = model_from_json(loaded_model_json)
         
        # load weights into new model
        loaded_model.load_weights(h5filePath)

        return loaded_model
    except:
        return None

def _tf_fspecial_gauss(size, sigma):
    """Function to mimic the 'fspecial' gaussian MATLAB function
    """
    x_data, y_data = np.mgrid[-size//2 + 1:size//2 + 1, -size//2 + 1:size//2 + 1]

    x_data = np.expand_dims(x_data, axis=-1)
    x_data = np.expand_dims(x_data, axis=-1)

    y_data = np.expand_dims(y_data, axis=-1)
    y_data = np.expand_dims(y_data, axis=-1)

    x = tf.constant(x_data, dtype=tf.float32)
    y = tf.constant(y_data, dtype=tf.float32)

    g = tf.exp(-((x**2 + y**2)/(2.0*sigma**2)))
    return g / tf.reduce_sum(g)

def tf_ssim(img1, img2, cs_map=False, mean_metric=True, size=11, sigma=1.5):
    window = _tf_fspecial_gauss(size, sigma) # window shape [size, size]
    K1 = 0.01
    K2 = 0.03
    L = 1  # depth of image (255 in case the image has a differnt scale)
    C1 = (K1*L)**2
    C2 = (K2*L)**2
    mu1 = tf.nn.conv2d(img1, window, strides=[1,1,1,1], padding='VALID')
    mu2 = tf.nn.conv2d(img2, window, strides=[1,1,1,1],padding='VALID')
    mu1_sq = mu1*mu1
    mu2_sq = mu2*mu2
    mu1_mu2 = mu1*mu2
    sigma1_sq = tf.nn.conv2d(img1*img1, window, strides=[1,1,1,1],padding='VALID') - mu1_sq
    sigma2_sq = tf.nn.conv2d(img2*img2, window, strides=[1,1,1,1],padding='VALID') - mu2_sq
    sigma12 = tf.nn.conv2d(img1*img2, window, strides=[1,1,1,1],padding='VALID') - mu1_mu2
    if cs_map:
        value = (((2*mu1_mu2 + C1)*(2*sigma12 + C2))/((mu1_sq + mu2_sq + C1)*
                    (sigma1_sq + sigma2_sq + C2)),
                (2.0*sigma12 + C2)/(sigma1_sq + sigma2_sq + C2))
    else:
        value = ((2*mu1_mu2 + C1)*(2*sigma12 + C2))/((mu1_sq + mu2_sq + C1)*
                    (sigma1_sq + sigma2_sq + C2))

    if mean_metric:
        value = tf.reduce_mean(value)
    return value

def tf_ms_ssim(img1, img2, mean_metric=True, level=5):
    weight = tf.constant([0.0448, 0.2856, 0.3001, 0.2363, 0.1333], dtype=tf.float32)
    mssim = []
    mcs = []
    for l in range(level):
        ssim_map, cs_map = tf_ssim(img1, img2, cs_map=True, mean_metric=False)
        mssim.append(tf.reduce_mean(ssim_map))
        mcs.append(tf.reduce_mean(cs_map))
        filtered_im1 = tf.nn.avg_pool(img1, [1,2,2,1], [1,2,2,1], padding='SAME')
        filtered_im2 = tf.nn.avg_pool(img2, [1,2,2,1], [1,2,2,1], padding='SAME')
        img1 = filtered_im1
        img2 = filtered_im2

    # list to tensor of dim D+1
    mssim = tf.stack(mssim, axis=0)
    mcs = tf.stack(mcs, axis=0)

    value = (tf.reduce_prod(mcs[0:level-1]**weight[0:level-1])*
                            (mssim[level-1]**weight[level-1]))

    if mean_metric:
        value = tf.reduce_mean(value)
    return value

def dssim_objective(y_pred,y_true):
    ssim = tf_ssim(y_pred,y_true)
    return (1 - ssim)/2.
    

In [27]:
def get_gated_connections(gatePercentageFactor,inputLayer):
    fractionG = Lambda(lambda x, factor : factor * x, arguments = {'factor': gatePercentageFactor})(inputLayer)
    complement = Lambda(lambda x: x[0] - x[1])([inputLayer,fractionG])
    
    return fractionG,complement

#x is conv layer
#y is de-conv layer
#gf is gating factor
#fg is fractional input from gate
#c is complement ie remaining fraction from the gate
#jt joining tensor of convolution layer and previous de-conv layer 

def get_cnn_dsc_architecture(model_path=None):
    
    if(model_path is None):
        sym_autoencoder = None
    else:
        sym_autoencoder = read_model_json(model_path[0],model_path[1])
    conv_filter = (3 , 3)
    if(sym_autoencoder is None):
        input_img = Input(shape=(None,None,1), name = "Image_input")  # adapt this if using `channels_first` image data format
        x1 = Conv2D(64, conv_filter, activation='relu', padding='same')(input_img)
        fg1,c1 = get_gated_connections(0.1,x1)
        
        x = MaxPooling2D((2, 2), padding='same')(fg1)
        x2 = Conv2D(64, conv_filter, activation='relu', padding='same')(x) 
        fg2,c2 = get_gated_connections(0.2,x2)

        x = MaxPooling2D((2, 2), padding='same')(fg2)
        x3 = Conv2D(128, conv_filter, activation='relu', padding='same')(x) 
        fg3,c3 = get_gated_connections(0.3,x3)

        x = MaxPooling2D((2, 2), padding='same')(x3)
        x4 = Conv2D(256, conv_filter, activation='relu', padding='same')(x) 
        fg4,c4 = get_gated_connections(0.4,x4)

        x = MaxPooling2D((2, 2), padding='same')(x4)
        x5 = Conv2D(512, conv_filter, activation='relu', padding='same')(x) 

        x = UpSampling2D((2, 2))(x5)
        y1 = Conv2DTranspose(256, conv_filter, activation='relu', padding='same')(x) 
        jt4 = Add()([y1,c4])
        x = UpSampling2D((2, 2))(jt4)

        y2 = Conv2DTranspose(128, conv_filter, activation='relu', padding='same')(x) 
        jt3 = Add()([y2,c3])
        x = UpSampling2D((2, 2))(jt3)

        y3 = Conv2DTranspose(64, conv_filter, activation='relu', padding='same')(x) 
        jt2 = Add()([y3,c2])
        x = UpSampling2D((2, 2))(jt2)

        jt1 = Add()([x,c1])
        y4 = Conv2DTranspose(64, conv_filter, activation='relu', padding='same')(jt1)
        y5 = Conv2DTranspose(1, conv_filter, activation='relu', padding='same')(y4) 

        layers = y5

        sym_autoencoder = Model(input_img,layers)
        sym_autoencoder.compile(optimizer='adam', loss = dssim_objective, metrics = ['accuracy',dssim_objective])
        print "Model created"
    else:
        print "Saved model loaded"
    print sym_autoencoder.summary()
    return sym_autoencoder

In [23]:
def get_gated_connections(gatePercentageFactor,inputLayer):
    fractionG = Lambda(lambda x, factor : factor * x, arguments = {'factor': gatePercentageFactor})(inputLayer)
    complement = Lambda(lambda x: x[0] - x[1])([inputLayer,fractionG])
    
    return fractionG,complement

#x is conv layer
#y is de-conv layer
#gf is gating factor
#fg is fractional input from gate
#c is complement ie remaining fraction from the gate
#jt joining tensor of convolution layer and previous de-conv layer 

def get_dil_cnn_dsc_architecture(model_path=None):
    
    if(model_path is None):
        sym_autoencoder = None
    else:
        sym_autoencoder = read_model_json(model_path[0],model_path[1])
    if(sym_autoencoder is None):
        input_img = Input(shape=(None,None,1), name = "Image_input")  # adapt this if using `channels_first` image data format
        x1 = Conv2D(64, (4, 4), activation='relu', padding='same')(input_img)
        fg1,c1 = get_gated_connections(0.1,x1)
        
        x = MaxPooling2D((2, 2), padding='same')(fg1)
        x2 = Conv2D(64, (4, 4), activation='relu', padding='same')(x) 
        fg2,c2 = get_gated_connections(0.2,x2)

        x = MaxPooling2D((2, 2), padding='same')(fg2)
        x3 = Conv2D(128, (4, 4), activation='relu', padding='same')(x) 
        fg3,c3 = get_gated_connections(0.3,x3)

        #x = Conv2D(128, (4, 4), activation='relu', padding='same',dilation_rate=(2,2))(x3)
        x4 = Conv2D(256, (4, 4), activation='relu', padding='same',dilation_rate=(2,2))(x3) 
        fg4,c4 = get_gated_connections(0.4,x4)

        #x = Conv2D(256, (4, 4), activation='relu', padding='same',dilation_rate=(4,4))(x4)
        x5 = Conv2D(512, (4, 4), activation='relu', padding='same',dilation_rate=(4,4))(x4) 

        #x = UpSampling2D((2, 2))(x5)
        y1 = Conv2DTranspose(256, (4, 4), activation='relu', padding='same')(x5) 
        jt4 = Add()([y1,c4])
        #x = UpSampling2D((2, 2))(jt4)

        y2 = Conv2DTranspose(128, (4, 4), activation='relu', padding='same')(jt4) 
        jt3 = Add()([y2,c3])
        x = UpSampling2D((2, 2))(jt3)

        y3 = Conv2DTranspose(64, (4, 4), activation='relu', padding='same')(x) 
        jt2 = Add()([y3,c2])
        x = UpSampling2D((2, 2))(jt2)

        jt1 = Add()([x,c1])
        y4 = Conv2DTranspose(64, (4, 4), activation='relu', padding='same')(jt1)
        y5 = Conv2DTranspose(1, (4, 4), activation='relu', padding='same')(y4) 

        layers = y5

        sym_autoencoder = Model(input_img,layers)
        sym_autoencoder.compile(optimizer='adam', loss = dssim_objective, metrics = ['accuracy',dssim_objective])
        print "Model created"
    else:
        print "Saved model loaded"
    print sym_autoencoder.summary()
    return sym_autoencoder

In [28]:
sym_autoencoder = get_cnn_dsc_architecture()

Model created
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
Image_input (InputLayer)        (None, None, None, 1 0                                            
__________________________________________________________________________________________________
conv2d_6 (Conv2D)               (None, None, None, 6 640         Image_input[0][0]                
__________________________________________________________________________________________________
lambda_9 (Lambda)               (None, None, None, 6 0           conv2d_6[0][0]                   
__________________________________________________________________________________________________
max_pooling2d_5 (MaxPooling2D)  (None, None, None, 6 0           lambda_9[0][0]                   
_______________________________________________________________________________________________

In [49]:
plot_model(sym_autoencoder,to_file='dilated_conv1.png',show_shapes=True)

In [85]:
model_checkpoint1 = ModelCheckpoint('./models/scaled_data_gated_cnn_autoencoder_oct_5layers_relu_adam_500E_8B_loss_msssim_2skip.hdf5',monitor = 'loss', verbose = 1,save_best_only=True)

In [30]:
sym_autoencoder.fit(x_train_noisy, x_train,
                epochs = 500,
                batch_size = 8,
                shuffle=True,
                validation_data=(x_test_noisy, x_test),
                callbacks=[TensorBoard(log_dir='/tmp/gated_cnn_autoencoder', 
                                       histogram_freq=0,
                                       write_graph=True),model_checkpoint1])

Train on 979 samples, validate on 109 samples
Epoch 1/500
976/979 [============================>.] - ETA: 0s - loss: 0.0972 - acc: 0.0000e+00 - dssim_objective: 0.0972
Epoch 00001: loss improved from inf to 0.09717, saving model to ./models/scaled_data_gated_cnn_autoencoder_oct_5layers_relu_adam_500E_8B_loss_msssim_2skip.hdf5
979/979 [==============================] - 211s 215ms/step - loss: 0.0972 - acc: 0.0000e+00 - dssim_objective: 0.0972 - val_loss: 0.0837 - val_acc: 0.0000e+00 - val_dssim_objective: 0.0837
Epoch 2/500
976/979 [============================>.] - ETA: 0s - loss: 0.0801 - acc: 0.0000e+00 - dssim_objective: 0.0801
Epoch 00002: loss improved from 0.09717 to 0.08013, saving model to ./models/scaled_data_gated_cnn_autoencoder_oct_5layers_relu_adam_500E_8B_loss_msssim_2skip.hdf5
979/979 [==============================] - 214s 219ms/step - loss: 0.0801 - acc: 0.0000e+00 - dssim_objective: 0.0801 - val_loss: 0.0813 - val_acc: 0.0000e+00 - val_dssim_objective: 0.0813
Epoch 3/

Epoch 19/500
976/979 [============================>.] - ETA: 0s - loss: 0.0658 - acc: 0.0000e+00 - dssim_objective: 0.0658
Epoch 00019: loss improved from 0.06651 to 0.06580, saving model to ./models/scaled_data_gated_cnn_autoencoder_oct_5layers_relu_adam_500E_8B_loss_msssim_2skip.hdf5
979/979 [==============================] - 208s 212ms/step - loss: 0.0658 - acc: 0.0000e+00 - dssim_objective: 0.0658 - val_loss: 0.0671 - val_acc: 0.0000e+00 - val_dssim_objective: 0.0671
Epoch 20/500
976/979 [============================>.] - ETA: 0s - loss: 0.0655 - acc: 0.0000e+00 - dssim_objective: 0.0655
Epoch 00020: loss improved from 0.06580 to 0.06545, saving model to ./models/scaled_data_gated_cnn_autoencoder_oct_5layers_relu_adam_500E_8B_loss_msssim_2skip.hdf5
979/979 [==============================] - 208s 212ms/step - loss: 0.0655 - acc: 0.0000e+00 - dssim_objective: 0.0655 - val_loss: 0.0658 - val_acc: 0.0000e+00 - val_dssim_objective: 0.0658
Epoch 21/500
976/979 [==========================

976/979 [============================>.] - ETA: 0s - loss: 0.0392 - acc: 0.0000e+00 - dssim_objective: 0.0392
Epoch 00053: loss improved from 0.03978 to 0.03917, saving model to ./models/scaled_data_gated_cnn_autoencoder_oct_5layers_relu_adam_500E_8B_loss_msssim_2skip.hdf5
979/979 [==============================] - 207s 212ms/step - loss: 0.0392 - acc: 0.0000e+00 - dssim_objective: 0.0392 - val_loss: 0.0409 - val_acc: 0.0000e+00 - val_dssim_objective: 0.0409
Epoch 54/500
976/979 [============================>.] - ETA: 0s - loss: 0.0390 - acc: 0.0000e+00 - dssim_objective: 0.0390
Epoch 00054: loss improved from 0.03917 to 0.03905, saving model to ./models/scaled_data_gated_cnn_autoencoder_oct_5layers_relu_adam_500E_8B_loss_msssim_2skip.hdf5
979/979 [==============================] - 208s 212ms/step - loss: 0.0391 - acc: 0.0000e+00 - dssim_objective: 0.0391 - val_loss: 0.0400 - val_acc: 0.0000e+00 - val_dssim_objective: 0.0400
Epoch 55/500
976/979 [============================>.] - ETA: 

Epoch 90/500
976/979 [============================>.] - ETA: 0s - loss: 0.0317 - acc: 0.0000e+00 - dssim_objective: 0.0317
Epoch 00090: loss improved from 0.03184 to 0.03173, saving model to ./models/scaled_data_gated_cnn_autoencoder_oct_5layers_relu_adam_500E_8B_loss_msssim_2skip.hdf5
979/979 [==============================] - 209s 214ms/step - loss: 0.0317 - acc: 0.0000e+00 - dssim_objective: 0.0317 - val_loss: 0.0336 - val_acc: 0.0000e+00 - val_dssim_objective: 0.0336
Epoch 91/500
976/979 [============================>.] - ETA: 0s - loss: 0.0315 - acc: 0.0000e+00 - dssim_objective: 0.0315
Epoch 00091: loss improved from 0.03173 to 0.03147, saving model to ./models/scaled_data_gated_cnn_autoencoder_oct_5layers_relu_adam_500E_8B_loss_msssim_2skip.hdf5
979/979 [==============================] - 210s 215ms/step - loss: 0.0315 - acc: 0.0000e+00 - dssim_objective: 0.0315 - val_loss: 0.0330 - val_acc: 0.0000e+00 - val_dssim_objective: 0.0330
Epoch 92/500
976/979 [==========================

Epoch 109/500
976/979 [============================>.] - ETA: 0s - loss: 0.0295 - acc: 0.0000e+00 - dssim_objective: 0.0295
Epoch 00109: loss improved from 0.02968 to 0.02949, saving model to ./models/scaled_data_gated_cnn_autoencoder_oct_5layers_relu_adam_500E_8B_loss_msssim_2skip.hdf5
979/979 [==============================] - 209s 213ms/step - loss: 0.0295 - acc: 0.0000e+00 - dssim_objective: 0.0295 - val_loss: 0.0307 - val_acc: 0.0000e+00 - val_dssim_objective: 0.0307
Epoch 110/500
976/979 [============================>.] - ETA: 0s - loss: 0.0296 - acc: 0.0000e+00 - dssim_objective: 0.0296
Epoch 00110: loss did not improve
979/979 [==============================] - 208s 213ms/step - loss: 0.0296 - acc: 0.0000e+00 - dssim_objective: 0.0296 - val_loss: 0.0313 - val_acc: 0.0000e+00 - val_dssim_objective: 0.0313
Epoch 111/500
976/979 [============================>.] - ETA: 0s - loss: 0.0292 - acc: 0.0000e+00 - dssim_objective: 0.0292
Epoch 00111: loss improved from 0.02949 to 0.02917, 

...

Epoch 368/500
976/979 [============================>.] - ETA: 0s - loss: 0.0212 - acc: 0.0000e+00 - dssim_objective: 0.0212
Epoch 00368: loss did not improve
979/979 [==============================] - 208s 213ms/step - loss: 0.0212 - acc: 0.0000e+00 - dssim_objective: 0.0212 - val_loss: 0.0241 - val_acc: 0.0000e+00 - val_dssim_objective: 0.0241
Epoch 369/500
976/979 [============================>.] - ETA: 0s - loss: 0.0211 - acc: 0.0000e+00 - dssim_objective: 0.0211
Epoch 00369: loss did not improve
979/979 [==============================] - 208s 213ms/step - loss: 0.0211 - acc: 0.0000e+00 - dssim_objective: 0.0211 - val_loss: 0.0227 - val_acc: 0.0000e+00 - val_dssim_objective: 0.0227
Epoch 370/500
976/979 [============================>.] - ETA: 0s - loss: 0.0213 - acc: 0.0000e+00 - dssim_objective: 0.0213
Epoch 00370: loss did not improve
979/979 [==============================] - 209s 214ms/step - loss: 0.0212 - acc: 0.0000e+00 - dssim_objective: 0.0212 - val_loss: 0.0233 - val_acc: 

Epoch 391/500
976/979 [============================>.] - ETA: 0s - loss: 0.0208 - acc: 0.0000e+00 - dssim_objective: 0.0208
Epoch 00391: loss did not improve
979/979 [==============================] - 210s 214ms/step - loss: 0.0208 - acc: 0.0000e+00 - dssim_objective: 0.0208 - val_loss: 0.0232 - val_acc: 0.0000e+00 - val_dssim_objective: 0.0232
Epoch 392/500
976/979 [============================>.] - ETA: 0s - loss: 0.0207 - acc: 0.0000e+00 - dssim_objective: 0.0207
Epoch 00392: loss did not improve
979/979 [==============================] - 210s 215ms/step - loss: 0.0207 - acc: 0.0000e+00 - dssim_objective: 0.0207 - val_loss: 0.0226 - val_acc: 0.0000e+00 - val_dssim_objective: 0.0226
Epoch 393/500
976/979 [============================>.] - ETA: 0s - loss: 0.0207 - acc: 0.0000e+00 - dssim_objective: 0.0207
Epoch 00393: loss did not improve
979/979 [==============================] - 210s 215ms/step - loss: 0.0207 - acc: 0.0000e+00 - dssim_objective: 0.0207 - val_loss: 0.0228 - val_acc: 

Epoch 414/500
976/979 [============================>.] - ETA: 0s - loss: 0.0206 - acc: 0.0000e+00 - dssim_objective: 0.0206
Epoch 00414: loss did not improve
979/979 [==============================] - 208s 213ms/step - loss: 0.0206 - acc: 0.0000e+00 - dssim_objective: 0.0206 - val_loss: 0.0227 - val_acc: 0.0000e+00 - val_dssim_objective: 0.0227
Epoch 415/500
976/979 [============================>.] - ETA: 0s - loss: 0.0204 - acc: 0.0000e+00 - dssim_objective: 0.0204
Epoch 00415: loss did not improve
979/979 [==============================] - 209s 213ms/step - loss: 0.0204 - acc: 0.0000e+00 - dssim_objective: 0.0204 - val_loss: 0.0232 - val_acc: 0.0000e+00 - val_dssim_objective: 0.0232
Epoch 416/500
976/979 [============================>.] - ETA: 0s - loss: 0.0205 - acc: 0.0000e+00 - dssim_objective: 0.0205
Epoch 00416: loss did not improve
979/979 [==============================] - 209s 213ms/step - loss: 0.0205 - acc: 0.0000e+00 - dssim_objective: 0.0205 - val_loss: 0.0228 - val_acc: 

Epoch 437/500
976/979 [============================>.] - ETA: 0s - loss: 0.0203 - acc: 0.0000e+00 - dssim_objective: 0.0203
Epoch 00437: loss did not improve
979/979 [==============================] - 209s 213ms/step - loss: 0.0202 - acc: 0.0000e+00 - dssim_objective: 0.0202 - val_loss: 0.0229 - val_acc: 0.0000e+00 - val_dssim_objective: 0.0229
Epoch 438/500
976/979 [============================>.] - ETA: 0s - loss: 0.0202 - acc: 0.0000e+00 - dssim_objective: 0.0202
Epoch 00438: loss did not improve
979/979 [==============================] - 208s 213ms/step - loss: 0.0202 - acc: 0.0000e+00 - dssim_objective: 0.0202 - val_loss: 0.0224 - val_acc: 0.0000e+00 - val_dssim_objective: 0.0224
Epoch 439/500
976/979 [============================>.] - ETA: 0s - loss: 0.0204 - acc: 0.0000e+00 - dssim_objective: 0.0204
Epoch 00439: loss did not improve
979/979 [==============================] - 209s 213ms/step - loss: 0.0204 - acc: 0.0000e+00 - dssim_objective: 0.0204 - val_loss: 0.0226 - val_acc: 

Epoch 460/500
976/979 [============================>.] - ETA: 0s - loss: 0.0196 - acc: 0.0000e+00 - dssim_objective: 0.0196
Epoch 00460: loss improved from 0.01979 to 0.01960, saving model to ./models/scaled_data_gated_cnn_autoencoder_oct_5layers_relu_adam_500E_8B_loss_msssim_2skip.hdf5
979/979 [==============================] - 208s 213ms/step - loss: 0.0196 - acc: 0.0000e+00 - dssim_objective: 0.0196 - val_loss: 0.0222 - val_acc: 0.0000e+00 - val_dssim_objective: 0.0222
Epoch 461/500
976/979 [============================>.] - ETA: 0s - loss: 0.0198 - acc: 0.0000e+00 - dssim_objective: 0.0198
Epoch 00461: loss did not improve
979/979 [==============================] - 208s 213ms/step - loss: 0.0198 - acc: 0.0000e+00 - dssim_objective: 0.0198 - val_loss: 0.0221 - val_acc: 0.0000e+00 - val_dssim_objective: 0.0221
Epoch 462/500
976/979 [============================>.] - ETA: 0s - loss: 0.0196 - acc: 0.0000e+00 - dssim_objective: 0.0196
Epoch 00462: loss did not improve
979/979 [=========

In [14]:
#Testing with gated model
def non_ref_img_denoise_metric(noisy_img,denoised_img):
    mni = noisy_img - denoised_img
    ssim_n = np.array(compare_ssim(noisy_img,mni,multichannel=True,full=True)[1])
    ssim_p = np.array(compare_ssim(noisy_img,denoised_img,multichannel=True,full=True)[1])
    
    return pearsonr(ssim_n.flatten(),ssim_p.flatten())


In [148]:
sym_autoencoder = load_model('./models/oct_epoch_500_batch_20/simple_cnn_oct.hdf5',custom_objects={'dssim_objective':dssim_objective})

In [149]:
#Read images from folder and save with along with names
def read_images_from_folder_with_names(folder_path):
    images = []
    for filename in os.listdir(folder_path):
        img = cv2.imread(os.path.join(folder_path,filename),0)
        if img is not None:
            images.append((filename,img))
    return images

In [150]:
#Read required images for testing
directory_img = './test_data/cropped_noisy/'
test_image_set = read_images_from_folder_with_names(directory_img)

In [151]:
#Reading the ground truth

gt = cv2.imread('./test_data/cropped5.tif',0)
gt = np.array(gt)
shape = list(gt.shape)
print shape
#Calculating nearest multiple of 16
shape[0] -= shape[0]%16
shape[1] -= shape[1]%16
print shape
gt = cv2.resize(gt,(shape[1],shape[0]),interpolation = cv2.INTER_CUBIC)
gt =[gt]
gt = np.array(gt,dtype=np.float32)
gt = gt/255.
gt = gt.reshape((shape[0],shape[1],1))
gt.shape

[280, 900]
[272, 896]


(272, 896, 1)

In [152]:
#Denoise images in test_image_set
for idx,test_image in enumerate(test_image_set):
    if len(test_image[0]) >=30:
        continue
    shape = list(test_image[1].shape)
    shape[0] -= shape[0]%16
    shape[1] -= shape[1]%16
    test_image_norm = cv2.resize(test_image[1],(shape[1],shape[0]),interpolation = cv2.INTER_CUBIC)
    
    test_image_norm = [test_image_norm]
    test_image_norm = np.array(test_image_norm,dtype=np.float32)
    test_image_norm = test_image_norm/255.
    
    test_image_norm = test_image_norm.reshape((test_image_norm.shape[0],test_image_norm.shape[1],test_image_norm.shape[2],1))
    out_image = sym_autoencoder.predict(test_image_norm,verbose=0)

    psnr = compare_psnr(gt,out_image[0])
    ssim = compare_ssim(gt,out_image[0],multichannel=True)

    print test_image[0][:-3]
    print psnr,ssim
   
    #cv2.imwrite(directory_img
     #           +test_image[0][:-3] + '_'+str(round(psnr,4))+'_'+str(round(ssim,5))+'.tif',out_image[0]*255.)
    

oct_shape_8_scale_7.
28.4883680046 0.903410185766
oct_shape_6_scale_6.
30.2538533636 0.918143100991
oct_shape_4_scale_3.
31.6469150379 0.934199916509
oct_shape_4_scale_4.
31.3342209711 0.930355452345
oct_shape_3_scale_2.
31.3653709425 0.938350928147


In [153]:
#Testing with simple model

In [154]:
model = load_model("./models/oct_epoch_500_batch_20/simple_cnn_oct.hdf5")

In [13]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, None, None, 1)     0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, None, None, 64)    640       
_________________________________________________________________
average_pooling2d_1 (Average (None, None, None, 64)    0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, None, None, 64)    36928     
_________________________________________________________________
average_pooling2d_2 (Average (None, None, None, 64)    0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, None, None, 64)    36928     
_________________________________________________________________
up_sampling2d_1 (UpSampling2 (None, None, None, 64)    0         
__________

In [110]:
#Adding Gamma noise
test_image = cv2.imread('./test_data/cropped5.tif',0)
print test_image.shape
#test_image = cv2.resize(test_image,(900,448),interpolation = cv2.INTER_CUBIC)
#print test_image.shape
gamma_param = [(3,2),(4,3),(4,4),(6,6),(8,7)]
for param in gamma_param:
    shape = param[0]
    scale = param[1]
    noise = np.random.gamma(shape,scale,test_image.shape)
    
    noisy_img = noise + test_image
    
    #Scaling
    noisy_img = 255.*((noisy_img - noisy_img.min())/(noisy_img.max()-noisy_img.min()))
    
    noisy_img = noisy_img.astype('uint8')
    
    noisy_img = np.clip(noise + test_image,0.,255.)
    
    cv2.imwrite('./test_data/cropped_noisy/oct_shape_'+str(shape)+'_scale_'+str(scale)+'.tif',noisy_img)

(280, 900)


In [43]:
#Testing on new data
path = './synthetic/4/'
test_image = cv2.imread(path+'test.tif',0)
shape = list(test_image.shape)
print shape
shape[0] -= shape[0]%16
shape[1] -= shape[1]%16
print shape
test_image = cv2.resize(test_image,(shape[1],shape[0]),interpolation = cv2.INTER_CUBIC)
test_image =[test_image]
test_image = np.array(test_image,dtype=np.float32)
test_image = test_image/255.
print test_image.shape
test_image = test_image.reshape((1,shape[0],shape[1],1))
print test_image.shape

[450, 900]
[448, 896]
(1, 448, 896)
(1, 448, 896, 1)


In [45]:
#Denoising the test_image
den_test_image = sym_autoencoder.predict(test_image)

In [ ]:
#Saving the denoised image
cv2.imwrite('denoised.png',den_test_image[0])